## Running multiple ML models at the same time
This notebook is an example of how to run multiple models side-by-side and combine results of all models. A video stream from a video source is processed by the hand, face, and person detection models. Combined result is then displayed.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option. 

You also need to specify your cloud API access token in [env.ini](../../env.ini) file, located in the same directory as this notebook.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

#### Specify where you want to run your inferences, model_zoo_url, model names for hand, face, and person detection, and video source

In [ ]:
# hw_location: where you want to run inference
#     @cloud to use DeGirum cloud
#     @local to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# hand_det_model_name: name of the model for hand detection
# face_det_model_name: name of the model for face detection
# person_det_model_name: name of the model for person detection
# video_source: video source for inference
#     camera index for local web camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
hw_location = "@cloud"
model_zoo_url = "https://cs.degirum.com/degirum/public"
hand_det_model_name = "yolo_v5s_hand_det--512x512_quant_n2x_orca1_1"
face_det_model_name = "yolo_v5s_face_det--512x512_quant_n2x_orca1_1"
person_det_model_name = "yolo_v5s_person_det--512x512_quant_n2x_orca1_1"
video_source = "../../images/person_face_hand.mp4"

#### The rest of the cells below should run without any modifications

In [ ]:
import degirum as dg, degirum_tools

# configure for Google Colab
degirum_tools.configure_colab()
# connect to AI inference engine getting token from env.ini file
zoo = dg.connect(hw_location, model_zoo_url, degirum_tools.get_token())
# load models for hand, face, and person detection
hand_det_model = zoo.load_model(hand_det_model_name)
face_det_model = zoo.load_model(face_det_model_name)
person_det_model = zoo.load_model(person_det_model_name, overlay_line_width=1)
# AI prediction loop, press 'x' or 'q' to stop video
with degirum_tools.Display("Hands, Faces and Persons") as display:
    for inference_result in degirum_tools.predict_stream(
        degirum_tools.CombiningCompoundModel(
            degirum_tools.CombiningCompoundModel(hand_det_model, face_det_model),
            person_det_model,
        ),
        video_source,
    ):
        display.show(inference_result)